In [4]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [5]:
# generate fake products name
def generate_fake_products_name():
    fake = Faker()
    return fake.bs().title()

# generate fake products price
def generate_fake_products_price():
    return random.randint(1, 100)

# generate fake products quantity
def generate_fake_products_quantity():
    return random.randint(1, 250)

# generate products sku units ratio
def generate_fake_products_sku_units_ratio():
    return random.randint(1, 24)

# generate fake description
def generate_fake_description():
    fake = Faker()
    description = fake.text()
    if description.count('.') > 1:
        return description.title()
    else:
        raise ValueError("Generated text is not a paragraph.")

# generate fake brand
def generate_fake_brand():
    fake = Faker()
    return fake.company()

# generate fake weight
def generate_fake_weight():
    return round(random.uniform(0.1, 15.0), 2)

# generate fake dimensions
def generate_fake_dimensions():
    return f"{random.randint(1, 100)}x{random.randint(1, 100)}x{random.randint(1, 100)}"

# generate fake date added
def generate_fake_date_added():
    fake = Faker()
    return fake.date()

# generate fake image URL
def generate_fake_image_url():
    width = random.randint(100, 1000)
    height = random.randint(100, 1000)
    return f"https://picsum.photos/{width}/{height}"


In [6]:
def fake_products():
    fake_products = {
        "products_id": secrets.token_hex(36),
        "products_name": generate_fake_products_name(),
        "products_price": generate_fake_products_price(),
        "products_quantity": generate_fake_products_quantity(),
        "products_sku_units_ratio": generate_fake_products_sku_units_ratio(),
        "description": generate_fake_description(),
        "brand": generate_fake_brand(),
        "weight": generate_fake_weight(),
        "dimensions": generate_fake_dimensions(),
        "date_added": generate_fake_date_added(),
        "image_url": generate_fake_image_url()
    }
    
    return fake_products

def multiple_fake_products(n):
    fake_products_data = [fake_products() for _ in range(n)]
    return json.dumps(fake_products_data, indent=4)

In [7]:
# insert fake products data into database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS products (
        products_id TEXT PRIMARY KEY NOT NULL,
        products_name TEXT NOT NULL,
        products_price REAL NOT NULL,
        products_quantity INTEGER NOT NULL,
        products_sku_units_ratio INTEGER NOT NULL,
        description TEXT NOT NULL,
        brand TEXT NOT NULL,
        weight REAL NOT NULL,
        dimensions TEXT NOT NULL,
        date_added DATE NOT NULL,
        image_url TEXT NOT NULL
    )
    '''
)

json_data = multiple_fake_products(350)

# save the json data as json
with open('json_data/products.json', 'w') as f:
    f.write(json_data)
    
for products in json.loads(json_data):
    c.execute(
        '''
        INSERT INTO products (products_id, products_name, products_price, products_quantity, products_sku_units_ratio, description, brand, weight, dimensions, date_added, image_url)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            products['products_id'],
            products['products_name'],
            products['products_price'],
            products['products_quantity'],
            products['products_sku_units_ratio'],
            products['description'],
            products['brand'],
            products['weight'],
            products['dimensions'],
            products['date_added'],
            products['image_url']
        )
    )
    
conn.commit()
conn.close() # must close the connection after commiting the changes